In [ ]:
!pip install plotly==5.2.2

## Import des librairies ##

In [ ]:
#Imports librairies de base
import numpy as np
import pandas as pd

#Bibliothèques pour Cross validation

#Affichage de HTML
from IPython.display import display, HTML
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 1. Lecture des données ##

Chargement et affichage des données d'apprentissage (Data/passagers.csv) dans un DataFrame nommé train.

In [ ]:
#Lecture du fichier passagers avec label "Survived", pour le training
train = pd.read_csv('drive/My Drive/EPSI/Data/passagers.csv')
train.sample(10)

*Informations des deux DataFrames train et Test.

In [ ]:
# Infos sur train
display(HTML('<span style="color:blue;font-weight:bold">train :</span>'))
train.info()

## 2. Analyse des features

### Analyse unidimensionnelle Age

In [ ]:
fig = px.box(train, x="Age",orientation='h')
fig.show() 


In [ ]:
x_data = ['1ère Classe', '2ème Classe','3ème Classe']


y_data = []
for c in range(3):
  y_data.append(train.query(f"Pclass=={c+1}").Age)

colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)', 'rgba(44, 160, 101, 0.5)']

fig = go.Figure()

for classe, age, couleur in zip(x_data, y_data, colors):
        fig.add_trace(go.Box(
            y=age,
            name=classe,
            boxpoints='all', # Montrer tous les points
            jitter=0.3,      # épaisseur représentation des points
            whiskerwidth=0.2,# épaisseur des traits
            fillcolor=couleur,
            marker_size=2,
            line_width=1)
        )

fig.update_layout(
    title='Age des passagers par Classe',
    yaxis=dict(
        autorange=True, # Valeurs de l'axe en fonction des données
        showgrid=True,  # Affichage de la grille
        dtick=5,        # Précision Axe
        gridcolor='rgb(255, 255, 255)', # Couleur Grille
        gridwidth=2,                    # Taille Grille
    )
)

fig.show()

### Distribution de l'age et Normale

In [ ]:
import plotly.figure_factory as ff # Utilisation de la méthode create_distplot

ages = train[~np.isnan(train.Age)].Age # Les valeurs inconnues (NaN sont éliminées)
hist_data = [ages]
fig = ff.create_distplot(hist_data, ['Distribution Age'], curve_type='normal', bin_size=5) # Distribution par tranche de 5 ans
fig.show()

### Effet de la Classe

In [ ]:
display(HTML('<span style="color:blue;font-weight:bold">Effet de la classe</span>'))
agg_classe = train[['Pclass', 'Survived']].groupby(['Pclass'])
effet_classe = pd.concat([agg_classe.count(),agg_classe.sum(),agg_classe.mean()],axis=1)
effet_classe.columns = ['nb_passagers','nb_survivants','taux_survie']

effet_classe

In [ ]:
# Utilisation de Plotly Express
libelles=['1ère Classe','2ème Classe','3ème Classe']

figpiepop = px.pie(effet_classe, values='nb_passagers', names=libelles, color =libelles, title='Répartition des passagers')
figpiepop.update_traces(textposition='inside', textinfo='percent+value+label')
figpiepop.show()
figpieresc = px.pie(effet_classe, values='nb_survivants', names=libelles, color =libelles, title='Répartition des survivants')
figpieresc.update_traces(textposition='inside', textinfo='percent+value+label')
figpieresc.show()
figbarsurvie = px.bar(effet_classe, x=libelles,y='taux_survie', labels={"x":"Classe", "taux_survie" :"Taux de Survie"},
                      title='Taux de survie', text=round(effet_classe.taux_survie*100,2))
figbarsurvie.show()

In [ ]:
# Utilisation de plotly.graph_objects
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "pie"}, {"type": "pie"}],[{"type": "bar", "colspan": 2}, None]],
    subplot_titles=("Répartition des passagers", "Répartition des survivants", "Taux de survie")
  )

fig.add_trace(
    go.Pie(labels=libelles, values=effet_classe.nb_passagers.values),
    row=1, col=1
)

fig.add_trace(
    go.Pie(labels=libelles, values=effet_classe.nb_survivants.values),
    row=1, col=2
)

fig.add_trace(
    go.Bar(x=libelles, y=effet_classe.taux_survie.values, showlegend=False,
           text=np.round(effet_classe.taux_survie.values*100,2).astype('str'),
           textposition='auto'),
    row=2, col=1
)

fig.update_layout(height=900, width=1000, title_text="Comparaison des classes dans le Titanic")
fig.update_traces(textposition='inside', textinfo='percent+value+label',selector={"type":"pie"})
fig.show()

## Analyse bidimensionnelle Iris

In [ ]:
df_iris = px.data.iris()
df_setosa = df_iris.query("species == 'setosa'")
# df_setosa[id] = df_setosa.index
df_setosa_orderby_sepal_width = df_setosa.sort_values('sepal_width').reset_index()
df_setosa_orderby_sepal_width['index'] = df_setosa_orderby_sepal_width.index
px.line(df_setosa_orderby_sepal_width,
              x='index', 
              y=['sepal_width','sepal_length'],
              title='Largeur / Longueur sépale setosa',
              labels={'index':"N° Iris"  ,'value':'Largeur'}
              )

### Animation

In [ ]:
tab_setosa_frames = []
for y in df_setosa_orderby_sepal_width["index"].unique():
    df = df_setosa_orderby_sepal_width[df_setosa_orderby_sepal_width["index"]<=y]
    df["frame"] = y
    tab_setosa_frames.append(df)
df_setosa_animation = pd.concat(tab_setosa_frames)
df_setosa_animation

In [ ]:
px.line(df_setosa_animation,
        x='index', 
        y=['sepal_width','sepal_length'],
        title='Largeur / Longueur sépale setosa',
        labels={'index':"N° Iris"  ,'value':'Largeur'},
        range_x=[0, 49], # ranges nécessaires car le graphique est d'abord affiché sans données
        range_y=[0, 6],
        animation_frame='frame'
        )

### Scatter Plot

In [ ]:
fig_basic=px.scatter(df_setosa, x="sepal_width", y="sepal_length",template ='plotly_white' )
fig_basic.show()

#### Identification de la tendance avec un bar Chart

In [ ]:
# Identification de la tendance avec un bar Chart
df_setosa['categ_sepal_width'] = pd.cut(df_setosa['sepal_width'],5)
df_groupby = df_setosa.groupby(['categ_sepal_width']).mean()
fig = px.bar(
    df_groupby,
    x=df_groupby.index.astype('str'),
    y='sepal_length',
    labels={'x':"Intervale Largeur Sépale"  ,'sepal_length':'Longeur Sépale'}
    )
fig.show()

#### Superposition de graphes (violin)

In [ ]:
fig_basic=px.scatter(df_setosa, x="sepal_width", y="sepal_length",template ='plotly_white' )
fig_violin=px.violin(df_setosa, x="sepal_width", y="sepal_length",template ='ggplot2' )
fig_basic.add_trace(fig_violin.data[0])
fig_basic.show()

#### Identification de la tendance avec une Trend Line

In [ ]:
def get_trendline_info(results):   
  ligne_resultat_r2 = results.px_fit_results.iloc[0].summary().tables[0][1]
  print(ligne_resultat_r2[2].data,':',ligne_resultat_r2 [3].data)
  tab_coefs = results.px_fit_results.iloc[0].summary().tables[1]
  print(f'y = {tab_coefs[2][1].data}x    + {tab_coefs[1][1].data}')

In [ ]:
fig = px.scatter(df_setosa, x="sepal_width", y="sepal_length", trendline="ols") # Ordinary Least Squares (OLS)
fig.show()
results = px.get_trendline_results(fig)
#print(results.px_fit_results.iloc[0].summary())
get_trendline_info(results)

In [ ]:
fig = px.scatter(df_iris, x="sepal_width", y="sepal_length", color="species", marginal_y="violin",
           marginal_x="box", trendline="ols", template="simple_white")
fig.show()

results = px.get_trendline_results(fig)
for specie in df_iris.species.unique():
  print(f'----------------\n    {specie}\n----------------')
  get_trendline_info(results.query(f"species == '{specie}'"))  